In [1]:
# Dependencies
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
import pymongo as pm

# Extract

In [2]:
# During data extraction, raw data is copied or exported from source locations

### Speed Dating Data


In [3]:
# Reading the csv file
csv_file = "resources/speed-dating.csv"
date_speed_df = pd.read_csv(csv_file)
date_speed_df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0,1,female,21.0,27.0,6,[4-6],asian/pacific islander/asian-american,european/caucasian-american,0,...,[0-3],[3-5],7.0,6.0,[6-8],[5-6],0.0,1,0,0
1,0,1,female,21.0,22.0,1,[0-1],asian/pacific islander/asian-american,european/caucasian-american,0,...,[0-3],[3-5],7.0,5.0,[6-8],[5-6],1.0,1,0,0
2,1,1,female,21.0,22.0,1,[0-1],asian/pacific islander/asian-american,asian/pacific islander/asian-american,1,...,[0-3],[3-5],7.0,NaN,[6-8],[0-4],1.0,1,1,1
3,0,1,female,21.0,23.0,2,[2-3],asian/pacific islander/asian-american,european/caucasian-american,0,...,[0-3],[3-5],7.0,6.0,[6-8],[5-6],0.0,1,1,1
4,0,1,female,21.0,24.0,3,[2-3],asian/pacific islander/asian-american,latino/hispanic american,0,...,[0-3],[3-5],6.0,6.0,[6-8],[5-6],0.0,1,1,1


### Reviews Data

In [4]:
reviews_df = pd.read_csv('resources/reviews.csv')
reviews_df.head()

,Unnamed: 0,Name,Review,Rating,#ThumbsUp,Date&Time,App
0,0,linah sibanda,On this app i cant find a partner,5,0,18-02-2022 01:19,Tinder
1,1,Norman Johnson,Tinder would be so much better if we could spe...,3,0,18-02-2022 01:16,Tinder
2,2,David Hume,Still doesn't correctly notify matches or mess...,1,0,18-02-2022 01:11,Tinder
3,3,Last 1 Standing,"Got banned because I updated my bio to say ""I ...",2,0,18-02-2022 01:11,Tinder
4,4,Arthur Magamedov,Love it!,5,0,18-02-2022 01:06,Tinder


### Data from website to be scrapped

In [5]:
# Read HTML from file
filepath = os.path.join("resources/template.html")
with open(filepath) as file:
    html = file.read()

In [6]:
# Create a Beautiful Soup Object
soup = bs(html, 'html.parser')

In [7]:
# Extract dating apps names and amount of their users from headlines
apps_and_users = soup.body.find_all('h2')[:24]
for t in apps_and_users:
    print(t.text)

1. Badoo — 480M Users
2. Tinder — 300M Users
3. Plenty of Fish — 150M Users
4. Bumble — 100 M Users
5. Adult Friend Finder — 99M Users
6. MenNation — 99M Users
7. Match — 96M Users
8. Ashley Madison — 65M Users
9. Happn — 50M Users
10. Zoosk — 40M Users
11. eharmony — 37M Users
12. OkCupid — 30M Users
13. Grindr — 27M Users
14. Coffee Meets Bagel — 21M Users
15. ChristianMingle — 16M Users
16. BeNaughty — 13.3M Users
17. EliteSingles — 13M Users
18. OurTime — 8.9M Users
19. BlackPeopleMeet — 5.7M Users
20. Hinge — 5.5M Users
21. HER — 4M Users
22. SilverSingles — 800,000 Users
23. LesbianPersonals — 500,000 Users
24. FriendFinder-X — 40,000 Users


# Transform
### Data cleansing and transformation

### Speed Dating Data

In [8]:
# Selecting relevant columns for future analysis
shortdata_df = date_speed_df [['gender', 'age', 'race','importance_same_race',
                               'field','importance_same_religion','attractive_o','sinsere_o','intelligence_o',
                               'funny_o','ambitous_o','shared_interests_o']].copy()
shortdata_df.head()

,gender,age,race,importance_same_race,field,importance_same_religion,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o
0,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,6.0,8.0,8.0,8.0,8.0,6.0
1,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,7.0,8.0,10.0,7.0,7.0,5.0
2,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,10.0,10.0,10.0,10.0,10.0,10.0
3,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,7.0,8.0,9.0,8.0,9.0,8.0
4,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,8.0,7.0,9.0,6.0,9.0,7.0


In [9]:
# Retrieve pattern base on the field
field_df = shortdata_df.loc[shortdata_df["importance_same_race"] >=10].groupby(["field"])["attractive_o"].count()
field_df

field
american studies                           9
business                                  19
clinical psychology                       17
ecology                                   20
education                                 17
education policy                          14
finance                                    5
higher ed. - m.a.                         18
international finance; economic policy    10
law                                       18
mba                                        9
psychology                                 9
social work                               78
Name: attractive_o, dtype: int64

In [10]:
# Counting the number of rows
len(shortdata_df)

8378

In [11]:
# Dropping all empty cells
reduced_df = shortdata_df.dropna()
reduced_df.reset_index(inplace=True)
reduced_df= reduced_df.drop('index', axis=1)
reduced_df

,gender,age,race,importance_same_race,field,importance_same_religion,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o
0,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,6.0,8.0,8.0,8.0,8.0,6.0
1,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,7.0,8.0,10.0,7.0,7.0,5.0
2,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,10.0,10.0,10.0,10.0,10.0,10.0
3,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,7.0,8.0,9.0,8.0,9.0,8.0
4,female,21.0,asian/pacific islander/asian-american,2.0,law,4.0,8.0,7.0,9.0,6.0,9.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6940,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,10.0,5.0,3.0,2.0,6.0,5.0
6941,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,6.0,3.0,7.0,3.0,7.0,2.0
6942,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,2.0,1.0,2.0,2.0,2.0,1.0
6943,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,5.0,7.0,5.0,5.0,3.0,6.0


In [12]:
# Renaming the columns for better understanding
renaming_df = reduced_df.rename(columns={"importance_same_race":"Same race",
                                         "importance_same_religion":"Same religion",
                                         "attractive_o":"Attractiveness",
                                         "sinsere_o":"Sincerity",
                                         "intelligence_o":"Intelligence",
                                         "gender":"Gender",
                                         "age":"Age",
                                         "race":"Race",
                                         "field":"Field",
                                         "funny_o":"Funny",
                                         "ambitous_o":"Ambition",
                                         "shared_interests_o":"Shared interests"})
renaming_df.tail()

,Gender,Age,Race,Same race,Field,Same religion,Attractiveness,Sincerity,Intelligence,Funny,Ambition,Shared interests
6940,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,10.0,5.0,3.0,2.0,6.0,5.0
6941,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,6.0,3.0,7.0,3.0,7.0,2.0
6942,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,2.0,1.0,2.0,2.0,2.0,1.0
6943,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,5.0,7.0,5.0,5.0,3.0,6.0
6944,male,25.0,european/caucasian-american,1.0,climate dynamics,1.0,8.0,8.0,7.0,7.0,7.0,7.0


### Reviews Data

In [13]:
# Basic information about the data
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681994 entries, 0 to 681993
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  681994 non-null  int64 
 1   Name        681989 non-null  object
 2   Review      680642 non-null  object
 3   Rating      681994 non-null  int64 
 4   #ThumbsUp   681994 non-null  int64 
 5   Date&Time   681994 non-null  object
 6   App         681994 non-null  object
dtypes: int64(3), object(4)
memory usage: 36.4+ MB


In [14]:
# Droping unwanted headers
reviews_df_clean = reviews_df.dropna(how='all').drop(['Unnamed: 0', '#ThumbsUp', 'Name', 'Review'], axis=1).copy()

In [15]:
# The colums needed
reviews_df_clean.head()

,Rating,Date&Time,App
0,5,18-02-2022 01:19,Tinder
1,3,18-02-2022 01:16,Tinder
2,1,18-02-2022 01:11,Tinder
3,2,18-02-2022 01:11,Tinder
4,5,18-02-2022 01:06,Tinder


In [16]:
# Needed information in a correct form
reviews_df_clean[['App', 'Rating', 'Date&Time']]

,App,Rating,Date&Time
0,Tinder,5,18-02-2022 01:19
1,Tinder,3,18-02-2022 01:16
2,Tinder,1,18-02-2022 01:11
3,Tinder,2,18-02-2022 01:11
4,Tinder,5,18-02-2022 01:06
...,...,...,...
681989,Hinge,2,12-07-2017 01:44
681990,Hinge,1,12-07-2017 01:36
681991,Hinge,5,12-07-2017 01:32
681992,Hinge,2,12-07-2017 01:28


In [17]:
# Seperating date and time
reviews_df_app_rating = pd.DataFrame(reviews_df_clean[['App', 'Rating', 'Date&Time']])
reviews_df_app_rating

,App,Rating,Date&Time
0,Tinder,5,18-02-2022 01:19
1,Tinder,3,18-02-2022 01:16
2,Tinder,1,18-02-2022 01:11
3,Tinder,2,18-02-2022 01:11
4,Tinder,5,18-02-2022 01:06
...,...,...,...
681989,Hinge,2,12-07-2017 01:44
681990,Hinge,1,12-07-2017 01:36
681991,Hinge,5,12-07-2017 01:32
681992,Hinge,2,12-07-2017 01:28


In [18]:
# Seperating the year from the date and time
reviews_df_app_rating['Year'] = pd.DatetimeIndex(reviews_df_app_rating['Date&Time']).year
reviews_df_app_rating

,App,Rating,Date&Time,Year
0,Tinder,5,18-02-2022 01:19,2022
1,Tinder,3,18-02-2022 01:16,2022
2,Tinder,1,18-02-2022 01:11,2022
3,Tinder,2,18-02-2022 01:11,2022
4,Tinder,5,18-02-2022 01:06,2022
...,...,...,...,...
681989,Hinge,2,12-07-2017 01:44,2017
681990,Hinge,1,12-07-2017 01:36,2017
681991,Hinge,5,12-07-2017 01:32,2017
681992,Hinge,2,12-07-2017 01:28,2017


### Data from https://www.datingadvice.com/online-dating/dating-sites-with-the-most-users

In [19]:
# Storing the data in a dataframe
data = {'App':['Badoo', 'Tinder', 'Plenty of Fish', 'Bumble', 'Adult Friend Finder','MenNation','Match','Ashley Madison','Happn','Zoosk','eharmony','OkCupid','Grindr','Coffee Meets Bagel','ChristianMingle','BeNaughty','EliteSingles','OurTime','BlackPeopleMeet','Hinge','HER','SilverSingles','LesbianPersonals','FriendFinder-X'], 'Users in million':[480, 300, 150, 100, 99, 99, 96, 65, 50, 40, 37, 30, 27, 21, 16, 13.3, 13, 8.9, 5.7, 5.5, 4, 0.8, 0.5, 0.04]}

# Create DataFrame
data_df = pd.DataFrame(data)
data_df

,App,Users in million
0,Badoo,480.00
1,Tinder,300.00
2,Plenty of Fish,150.00
3,Bumble,100.00
4,Adult Friend Finder,99.00
5,MenNation,99.00
6,Match,96.00
7,Ashley Madison,65.00
8,Happn,50.00
9,Zoosk,40.00


In [20]:
# Exporting data to CSV file
data_df.to_csv('resources/apps_and_users.csv', index = False)

# Load
### Exporting the data into MongoDB

In [21]:
# The default port used by MongoDB is 27017
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

### Speed Dating Data

In [22]:
# Creating the database and the collection
db = client.DatingDB
collection = db.love_finder

In [23]:
# Converting the dataframe into a dictionary
renaming_df.reset_index(drop=True)
data_dict = renaming_df.to_dict("records")
data_dict

[{'Gender': 'female',
  'Age': 21.0,
  'Race': 'asian/pacific islander/asian-american',
  'Same race': 2.0,
  'Field': 'law',
  'Same religion': 4.0,
  'Attractiveness': 6.0,
  'Sincerity': 8.0,
  'Intelligence': 8.0,
  'Funny': 8.0,
  'Ambition': 8.0,
  'Shared interests': 6.0},
 {'Gender': 'female',
  'Age': 21.0,
  'Race': 'asian/pacific islander/asian-american',
  'Same race': 2.0,
  'Field': 'law',
  'Same religion': 4.0,
  'Attractiveness': 7.0,
  'Sincerity': 8.0,
  'Intelligence': 10.0,
  'Funny': 7.0,
  'Ambition': 7.0,
  'Shared interests': 5.0},
 {'Gender': 'female',
  'Age': 21.0,
  'Race': 'asian/pacific islander/asian-american',
  'Same race': 2.0,
  'Field': 'law',
  'Same religion': 4.0,
  'Attractiveness': 10.0,
  'Sincerity': 10.0,
  'Intelligence': 10.0,
  'Funny': 10.0,
  'Ambition': 10.0,
  'Shared interests': 10.0},
 {'Gender': 'female',
  'Age': 21.0,
  'Race': 'asian/pacific islander/asian-american',
  'Same race': 2.0,
  'Field': 'law',
  'Same religion': 4.0,


In [24]:
# Loading the data into the collection
collection.insert_many(data_dict)

### Reviews Data

In [25]:
# Creating the database and the collection
db = client.DatingDB
collection = db.love_reviews

In [26]:
# Converting the dataframe into a dictionary
reviews_df_app_rating.reset_index(drop=True)
data_dictt = reviews_df_app_rating.to_dict("records")
data_dictt

[{'App': 'Tinder', 'Rating': 5, 'Date&Time': '18-02-2022 01:19', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 3, 'Date&Time': '18-02-2022 01:16', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 1, 'Date&Time': '18-02-2022 01:11', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 2, 'Date&Time': '18-02-2022 01:11', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 5, 'Date&Time': '18-02-2022 01:06', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 5, 'Date&Time': '18-02-2022 00:57', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 5, 'Date&Time': '18-02-2022 00:52', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 1, 'Date&Time': '18-02-2022 00:37', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 5, 'Date&Time': '18-02-2022 00:28', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 3, 'Date&Time': '18-02-2022 00:23', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 2, 'Date&Time': '18-02-2022 00:23', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 5, 'Date&Time': '18-02-2022 00:17', 'Year': 2022},
 {'App': 'Tinder', 'Rating': 1, 'Date&Ti

In [27]:
# Loading the data into the collection
collection.insert_many(data_dictt)

### Data from https://www.datingadvice.com/online-dating/dating-sites-with-the-most-users

In [28]:
# Creating the database and the collection
db = client.DatingDB
collection = db.popularity

In [29]:
# Converting the dataframe into a dictionary
data_df.reset_index(drop=True)
data_dicts = data_df.to_dict("records")
data_dicts

[{'App': 'Badoo', 'Users in million': 480.0},
 {'App': 'Tinder', 'Users in million': 300.0},
 {'App': 'Plenty of Fish', 'Users in million': 150.0},
 {'App': 'Bumble', 'Users in million': 100.0},
 {'App': 'Adult Friend Finder', 'Users in million': 99.0},
 {'App': 'MenNation', 'Users in million': 99.0},
 {'App': 'Match', 'Users in million': 96.0},
 {'App': 'Ashley Madison', 'Users in million': 65.0},
 {'App': 'Happn', 'Users in million': 50.0},
 {'App': 'Zoosk', 'Users in million': 40.0},
 {'App': 'eharmony', 'Users in million': 37.0},
 {'App': 'OkCupid', 'Users in million': 30.0},
 {'App': 'Grindr', 'Users in million': 27.0},
 {'App': 'Coffee Meets Bagel', 'Users in million': 21.0},
 {'App': 'ChristianMingle', 'Users in million': 16.0},
 {'App': 'BeNaughty', 'Users in million': 13.3},
 {'App': 'EliteSingles', 'Users in million': 13.0},
 {'App': 'OurTime', 'Users in million': 8.9},
 {'App': 'BlackPeopleMeet', 'Users in million': 5.7},
 {'App': 'Hinge', 'Users in million': 5.5},
 {'App': 

In [30]:
# Loading the data into the collection
collection.insert_many(data_dicts)

# Example of Analysis that could be performed

In [31]:
# Counting the gender of each race group in the dataset
racecount=renaming_df.groupby(['Race'])['Gender'].count()
racecount

Race
asian/pacific islander/asian-american    1674
black/african american                    365
european/caucasian-american              3888
latino/hispanic american                  565
other                                     453
Name: Gender, dtype: int64

In [32]:
# Retrieve pattern base on the field
field_df = renaming_df.loc[renaming_df["Same religion"] >=10].groupby(["Field"])["Attractiveness"].count()
field_df

Field
business                             13
economics; sociology                  9
education                            16
education policy                     13
elementary education - preservice    13
international affairs                19
law                                  17
ma science education                 17
marine geophysics                     3
mathematics                          19
neuroscience and education           16
operations research                  14
political science                    16
psychology                           45
social work                          39
speech pathology                     13
Name: Attractiveness, dtype: int64